In [1]:
import os
import mahotas as mh
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pickle
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 10 x 8 inches
plt.gray()

###Build training set

In [29]:
def create_cluster_script_build_training_set(species_names, sizes):

    new_script_file = '/cellar/users/ramarty/Projects/ants/scripts/cluster/version5.0/build_training.binary.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(species_names)))
        out_file.write("#$ -l h_vmem=1G\n")
        out_file.write("#$ -tc 4\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set species_names=({0})\n".format(" ".join(species_names)))
        out_file.write("set sizes=({0})\n".format(" ".join(sizes)))
        out_file.write("\n")

        out_file.write("set species_name=$species_names[$SGE_TASK_ID]\n")
        out_file.write("set size=$sizes[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("python /cellar/users/ramarty/Projects/ants/scripts/python/version5.0/build_training.binary.py $species_name $size\n")
        
        out_file.write("date\n")

In [30]:
species_names = ['cfellah', 'leptothorax', 'leptothorax']
sizes = ['30', '30', '100']

In [31]:
create_cluster_script_build_training_set(species_names, sizes)

###Train classifier

In [5]:
def create_cluster_script_train_classifier(species_names, sizes):

    new_script_file = '/cellar/users/ramarty/Projects/ants/scripts/cluster/version5.0/train_classifier.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(species_names)))
        out_file.write("#$ -l h_vmem=10G\n")
        out_file.write("#$ -tc 4\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set species_names=({0})\n".format(" ".join(species_names)))
        out_file.write("set sizes=({0})\n".format(" ".join(sizes)))
        out_file.write("\n")

        out_file.write("set species_name=$species_names[$SGE_TASK_ID]\n")
        out_file.write("set size=$sizes[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("python /cellar/users/ramarty/Projects/ants/scripts/python/version5.0/train_classifier.binary.py $species_name $size\n")
        
        out_file.write("date\n")

In [6]:
species_names = ['cfellah', 'cfellah', 'leptothorax', 'leptothorax', 'leptothorax']
sizes = ['50', '30', '100', '30', '50']

In [7]:
create_cluster_script_train_classifier(species_names, sizes)

### Real test photos

In [95]:
files = list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/photos/') if ('2017' in x) or ('2016' in x)]))[-5:]
files = list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/photos/') if ('2014' in x) or ('box101' in x)]))[-5:]

In [96]:
files

['box101-01440001',
 'box61-20140125-2110-01454402',
 'box101-01134001',
 'box61-20140119-1410-00367231',
 'box101-00799201']

###Run the classifiers on several photos

In [97]:
def binarize_labels(paths):
    new_script_file = '/cellar/users/ramarty/Projects/ants/scripts/cluster/version5.0/binarize_labels.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(paths)))
        out_file.write("#$ -l h_vmem=30G\n")
        out_file.write("#$ -tc 40\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("date\n")
        
        out_file.write("set paths=({0})\n".format(" ".join(paths)))
        out_file.write("\n")

        out_file.write("date\n")
        
        out_file.write("set p=$paths[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("python /cellar/users/ramarty/Projects/ants/scripts/python/version5.0/binarize_labels.py $p\n")
        
        out_file.write("date\n")

In [98]:
paths = ['/cellar/users/ramarty/Data/ants/photos/'+y+'.png' for y in list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/photos/') if ('2017' in x) or ('2016' in x)]))[-5:]] + ['/cellar/users/ramarty/Data/ants/photos/'+y+'.png' for y in list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/photos/') if ('2014' in x) or ('box101' in x)]))[-5:]]

In [99]:
paths

['/cellar/users/ramarty/Data/ants/photos/box61-20170511-0538-00273600.png',
 '/cellar/users/ramarty/Data/ants/photos/box41-20170318-0359-00100800.png',
 '/cellar/users/ramarty/Data/ants/photos/box41-20170322-1359-00864000.png',
 '/cellar/users/ramarty/Data/ants/photos/box41-20170323-0459-00972000.png',
 '/cellar/users/ramarty/Data/ants/photos/box51-20170512-0307-00432000.png',
 '/cellar/users/ramarty/Data/ants/photos/box101-01440001.png',
 '/cellar/users/ramarty/Data/ants/photos/box61-20140125-2110-01454402.png',
 '/cellar/users/ramarty/Data/ants/photos/box101-01134001.png',
 '/cellar/users/ramarty/Data/ants/photos/box61-20140119-1410-00367231.png',
 '/cellar/users/ramarty/Data/ants/photos/box101-00799201.png']

In [100]:
binarize_labels(paths)

In [110]:
def create_cluster_script_test_classifier(images, classifiers, outputs, sizes):

    new_script_file = '/cellar/users/ramarty/Projects/ants/scripts/cluster/version5.0/test_classifier.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(images)))
        out_file.write("#$ -l h_vmem=10G\n")
        out_file.write("#$ -tc 40\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set classifiers=({0})\n".format(" ".join(classifiers)))
        out_file.write("set images=({0})\n".format(" ".join(images)))
        out_file.write("set outputs=({0})\n".format(" ".join(outputs)))
        out_file.write("set sizes=({0})\n".format(" ".join(sizes)))
        out_file.write("\n")

        out_file.write("set classifier=$classifiers[$SGE_TASK_ID]\n")
        out_file.write("set image=$images[$SGE_TASK_ID]\n")
        out_file.write("set output=$outputs[$SGE_TASK_ID]\n")
        out_file.write("set size=$sizes[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("python /cellar/users/ramarty/Projects/ants/scripts/python/version5.0/Test.py -s $size $classifier $image $output\n")
        
        out_file.write("date\n")

In [143]:
species = 'leptothorax'

In [144]:
if species == 'cfellah':
    images = ['/cellar/users/ramarty/Data/ants/photos/'+y for y in list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/photos/') if ('2017' in x) or ('2016' in x)]))[-5:]]
else:
    images = ['/cellar/users/ramarty/Data/ants/photos/'+y for y in list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/photos/') if ('2014' in x) or ('box101' in x)]))[-5:]]
classifiers = ['~/Data/ants/version5.0/classifiers/{0}/'.format(species)+x for x in os.listdir('/cellar/users/ramarty/Data/ants/version5.0/classifiers/{0}/'.format(species)) if 'pkl' in x]
outputs = ['~/Data/ants/version5.0/predictions/{0}/'.format(species)]

In [145]:
images

['/cellar/users/ramarty/Data/ants/photos/box101-01440001',
 '/cellar/users/ramarty/Data/ants/photos/box61-20140125-2110-01454402',
 '/cellar/users/ramarty/Data/ants/photos/box101-01134001',
 '/cellar/users/ramarty/Data/ants/photos/box61-20140119-1410-00367221',
 '/cellar/users/ramarty/Data/ants/photos/box101-00799201']

In [121]:
classifiers

['~/Data/ants/version5.0/classifiers/cfellah/random_forest.30.pkl',
 '~/Data/ants/version5.0/classifiers/cfellah/random_forest.50.pkl']

In [122]:
all_images = []
for image in images:
    all_images.extend([image]*len(classifiers))
all_classifiers = classifiers*len(images)
all_outputs = outputs*len(all_classifiers)
all_sizes = [x.split('/')[-1].split('.')[1] for x in all_classifiers]

In [123]:
len(all_images), len(all_classifiers), len(all_outputs)

(10, 10, 10)

In [124]:
create_cluster_script_test_classifier(all_images, all_classifiers, all_outputs, all_sizes)

###Applying filters

In [39]:
def create_cluster_script_filtering(test_photos, species_names, image_sizes, kernel_sizes):

    new_script_file = '/cellar/users/ramarty/Projects/ants/scripts/cluster/version5.0/filter.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/ants/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-10\n")
        out_file.write("#$ -t 1-{0}\n".format(len(species_names)))
        out_file.write("#$ -l h_vmem=40G\n")
        out_file.write("#$ -tc 100\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set test_photos=({0})\n".format(" ".join(test_photos)))
        out_file.write("set species_names=({0})\n".format(" ".join(species_names)))
        out_file.write("set sizes=({0})\n".format(" ".join(image_sizes)))
        out_file.write("set kernels=({0})\n".format(" ".join(kernel_sizes)))
        out_file.write("\n")

        out_file.write("set test_photo=$test_photos[$SGE_TASK_ID]\n")
        out_file.write("set species_name=$species_names[$SGE_TASK_ID]\n")
        out_file.write("set size=$sizes[$SGE_TASK_ID]\n")
        out_file.write("set kernel=$kernels[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("python /cellar/users/ramarty/Projects/ants/scripts/python/version5.0/filtering.py $test_photo $species_name $size $kernel\n")
        
        out_file.write("date\n")

In [40]:
kernel_sizes = ['25', '50', '75', '100']

In [41]:
cfellah_test_files = [y for y in list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/photos/') if ('2017' in x) or ('2016' in x)]))[-5:]]
leptothorax_test_files = [y for y in list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/photos/') if ('2014' in x) or ('box101' in x)]))[-5:]]

In [46]:
leptothorax_test_files

['box101-01440001',
 'box61-20140125-2110-01454402',
 'box101-01134001',
 'box61-20140119-1410-00367221',
 'box101-00799201']

In [42]:
test_photos = cfellah_test_files*2 + leptothorax_test_files*3
species = ['cfellah' for x in range(10)] + ['leptothorax' for x in range(15)]
image_sizes = ['30' for x in range(5)] + ['50' for x in range(5)] + ['30' for x in range(5)] + ['50' for x in range(5)] + ['100' for x in range(5)]
len(test_photos), len(species), len(image_sizes)

(25, 25, 25)

In [43]:
all_test_photos, all_species, all_image_sizes, all_kernel_sizes = [], [], [], []
for kernel_size in kernel_sizes:
    all_test_photos.extend(test_photos)
    all_species.extend(species)
    all_image_sizes.extend(image_sizes)
    all_kernel_sizes.extend([kernel_size]*len(test_photos))

In [44]:
len(all_test_photos), len(all_species), len(all_image_sizes), len(all_kernel_sizes)

(100, 100, 100, 100)

In [45]:
create_cluster_script_filtering(all_test_photos, all_species, all_image_sizes, all_kernel_sizes)